In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Load data

In [ ]:
DatName='DYAD16NF'
Raw=pd.read_csv('/Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/'+DatName+'.csv', delimiter=';')
Raw

### Create an empty dataframe

In [3]:
# Initialize DFS with explicitly defined data types
DFS = pd.DataFrame({
    'frame_timestamp': pd.Series(dtype=float),
    'si_ry': pd.Series(dtype=float),
    'si_by': pd.Series(dtype=float),
    'si_rb': pd.Series(dtype=float)
})

DFS

Empty DataFrame
Columns: [frame_timestamp, si_ry, si_by, si_rb]
Index: []

In [4]:
# When loading the data for the first time, replace commas with dots and convert to float
Raw['Time'] = Raw['Time'].str.replace(',', '.').astype(float)

# Create an array with steps of 0,25 s until the max timestemp in Raw is reached
Seconds_025 = np.arange(0, np.max(Raw['Time']) + 0.25, 0.25)
Seconds_025

array([0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 6.0250e+02, 6.0275e+02,
       6.0300e+02])

In [5]:
# Replace all spaces with underscores for the entire dataset
Raw = Raw.replace(" ", "_", regex=True)
Raw

Time  Default Markers Social_bluered Social_blueyellow  \
0       0.0      NaN    Min0            NaN               NaN   
1       0.1      NaN     NaN            NaN               NaN   
2       0.2      NaN     NaN            NaN               NaN   
3       0.3      NaN     NaN            NaN               NaN   
4       0.4      NaN     NaN            NaN               NaN   
...     ...      ...     ...            ...               ...   
6026  602.6      NaN     NaN            NaN               NaN   
6027  602.7      NaN     NaN            NaN               NaN   
6028  602.8      NaN     NaN            NaN               NaN   
6029  602.9      NaN     NaN            NaN               NaN   
6030  603.0      NaN     NaN            NaN               NaN   

     Social_redyellow  Misc_bluered  Misc_blueyellow Misc_redyellow,  
0                 NaN           NaN              NaN               ,  
1                 NaN           NaN              NaN             NaN  
2                 NaN           NaN              NaN             NaN  
3                 NaN           NaN              NaN             NaN  
4                 NaN           NaN              NaN             NaN  
...               ...           ...              ...             ...  
6026       RY_distant           NaN              NaN             NaN  
6027       RY_distant           NaN              NaN             NaN  
6028       RY_distant           NaN              NaN             NaN  
6029       RY_distant           NaN              NaN             NaN  
6030       RY_distant           NaN              NaN               ,  

[6031 rows x 9 columns]

### Fill dataframe with values from raw data

While filling in the new DFS dataset by the 'raw' dataset, I need to merge rows of the 'raw' dataset. Column Time from the "raw" dataset  is in 0,1s steps, after merging   0.25 s intervals are to be between rows.    
If there are different values within the same column in the same interval, take the more prevalent one or if they are 50:50 pick one randomly. 

In [6]:
from collections import Counter

# Loop through each interval
for sec in Seconds_025:
    # Filter data for the current time range
    dd = Raw[(Raw['Time'] >= sec) & (Raw['Time'] < sec + 0.25)]
    
    # Initialize a row for the current interval
    row = {'frame_timestamp': sec}
    
    # Process each column (Social_bluered, Social_blueyellow, Social_redyellow)
    for column, new_column in zip(
        ['Social_bluered', 'Social_blueyellow', 'Social_redyellow'],
        ['si_rb', 'si_by', 'si_ry']
    ):
        if not dd[column].empty:
            # Count occurrences of each value in the interval
            counts = Counter(dd[column].dropna())
            if counts:
                # Get the most prevalent value
                most_common = counts.most_common(2)
                if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:
                    # If there’s a tie, pick randomly
                    row[new_column] = np.random.choice([most_common[0][0], most_common[1][0]])
                else:
                    # Otherwise, take the most common value
                    row[new_column] = most_common[0][0]
            else:
                row[new_column] = np.nan  # No valid data in this interval
    
    # Append the row to DFS
    DFS = pd.concat([DFS, pd.DataFrame([row])], ignore_index=True)

# Save to file
DFS.to_csv("/Users/ruzenkakaldenbach/Desktop/filled_intervals.csv", index=False)

# Display the final DataFrame
DFS


frame_timestamp       si_ry si_by si_rb
0                0.00         NaN   NaN   NaN
1                0.25         NaN   NaN   NaN
2                0.50         NaN   NaN   NaN
3                0.75         NaN   NaN   NaN
4                1.00         NaN   NaN   NaN
...               ...         ...   ...   ...
2408           602.00  RY_distant   NaN   NaN
2409           602.25  RY_distant   NaN   NaN
2410           602.50  RY_distant   NaN   NaN
2411           602.75  RY_distant   NaN   NaN
2412           603.00  RY_distant   NaN   NaN

[2413 rows x 4 columns]

Reduce information by replacing NaN with 0, _clear with 1, _distant with 2 and _unclear with 3.

In [8]:
# Replace NaN with 0
DFS.fillna(0, inplace=True)

# Replace string suffixes with numeric values
replacements = {
    '_clear': 1,
    '_distant': 2,
    '_unclear': 3
}

# Replace string suffixes in the relevant columns
for key, value in replacements.items():
    DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)

# Convert relevant columns to integers, preserve `frame_timestamp` as float
DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].astype(int)

# Save the reduced DataFrame
DFS.to_csv("/Users/ruzenkakaldenbach/Desktop/final_solomon.csv", index=False)

DFS

frame_timestamp  si_ry  si_by  si_rb
0                0.00      0      0      0
1                0.25      0      0      0
2                0.50      0      0      0
3                0.75      0      0      0
4                1.00      0      0      0
...               ...    ...    ...    ...
2408           602.00      2      0      0
2409           602.25      2      0      0
2410           602.50      2      0      0
2411           602.75      2      0      0
2412           603.00      2      0      0

[2413 rows x 4 columns]

### Loop over all spreadsheets

In [7]:
import pandas as pd
import numpy as np
from collections import Counter
import os

# List of dataset names
dataset_names = [
    "DYAD02F", "DYAD06F", "DYAD06NF", "DYAD10F", "DYAD10NF", "DYAD11F",
    "DYAD11NF", "DYAD12F", "DYAD12NF", "DYAD14F", "DYAD14NF", "DYAD15F",
    "DYAD15I", "DYAD15NF", "DYAD16F", "DYAD16I", "DYAD16NF", "DYAD18F",
    "DYAD18I", "DYAD18NF", "DYAD21F", "DYAD21NF", "DYAD23F", "DYAD23NF",
    "DYAD24F", "DYAD24NF"
]

# Base directory for input and output
input_dir = '/Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/'
output_dir = '/Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/'

# Loop through each dataset
for DatName in dataset_names:
    file_path = f"{input_dir}{DatName}.csv"
    
    # Skip processing if file does not exist
    if not os.path.exists(file_path):
        print(f"Skipping {DatName}: File not found at {input_dir}")
        continue

    # Load the dataset
    Raw = pd.read_csv(f'{input_dir}{DatName}.csv', delimiter=';')
    print(f"Processing {DatName}...")

    # Replace commas with dots and convert Time column to float
    Raw['Time'] = Raw['Time'].str.replace(',', '.').astype(float)

    # Replace all spaces with underscores for the entire dataset
    Raw = Raw.replace(" ", "_", regex=True)

    # Create the new DFS DataFrame
    DFS = pd.DataFrame({
        'frame_timestamp': pd.Series(dtype=float),
        'si_ry': pd.Series(dtype=float),
        'si_by': pd.Series(dtype=float),
        'si_rb': pd.Series(dtype=float)
    })

    # Create an array with steps of 0.25 seconds
    Seconds_025 = np.arange(0, np.max(Raw['Time']) + 0.25, 0.25)

    # Loop through each interval
    for sec in Seconds_025:
        # Filter data for the current time range
        dd = Raw[(Raw['Time'] >= sec) & (Raw['Time'] < sec + 0.25)]

        # Initialize a row for the current interval
        row = {'frame_timestamp': sec}

        # Process each column (Social_bluered, Social_blueyellow, Social_redyellow)
        for column, new_column in zip(
            ['Social_bluered', 'Social_blueyellow', 'Social_redyellow'],
            ['si_rb', 'si_by', 'si_ry']
        ):
            if not dd[column].empty:
                # Count occurrences of each value in the interval
                counts = Counter(dd[column].dropna())
                if counts:
                    # Get the most prevalent value
                    most_common = counts.most_common(2)
                    if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:
                        # If there’s a tie, pick randomly
                        row[new_column] = np.random.choice([most_common[0][0], most_common[1][0]])
                    else:
                        # Otherwise, take the most common value
                        row[new_column] = most_common[0][0]
                else:
                    row[new_column] = np.nan  # No valid data in this interval

        # Append the row to DFS
        DFS = pd.concat([DFS, pd.DataFrame([row])], ignore_index=True)

    # Replace NaN with 0
    DFS.fillna(0, inplace=True)

    # Replace string suffixes with numeric values
    replacements = {
        '_clear': 1,
        '_distant': 2,
        '_unclear': 3
    }

    # Replace string suffixes in the relevant columns
    for key, value in replacements.items():
        DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)
        # .astype(int) is to 

    # Convert relevant columns to integers, preserve `frame_timestamp` as float
    DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].astype(int)

    # Save the reduced DataFrame
    output_file = f'{output_dir}solomon_{DatName}.csv'
    DFS.to_csv(output_file, index=False)
    print(f"Saved {DatName} to {output_file}.")

print("Processing complete.")

Skipping DYAD02F: File not found at /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/
Skipping DYAD06F: File not found at /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/
Processing DYAD06NF...


/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_96496/2234130726.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)
/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_96496/2234130726.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)


Saved DYAD06NF to /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/solomon_DYAD06NF.csv.
Processing DYAD10F...


/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_96496/2234130726.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)
/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_96496/2234130726.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)


Saved DYAD10F to /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/solomon_DYAD10F.csv.
Processing DYAD10NF...


...

/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_96496/2234130726.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)
/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_96496/2234130726.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)


Saved DYAD21NF to /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/solomon_DYAD21NF.csv.
Skipping DYAD23F: File not found at /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/
Processing DYAD23NF...


/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_96496/2234130726.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)


Saved DYAD23NF to /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/solomon_DYAD23NF.csv.
Skipping DYAD24F: File not found at /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/
Processing DYAD24NF...
Saved DYAD24NF to /Users/ruzenkakaldenbach/Desktop/Behaviour/solomon/solomon_DYAD24NF.csv.
Processing complete.


/var/folders/m7/mw8cknkx6b58s7sq4mdg8nhc0000gn/T/ipykernel_96496/2234130726.py:92: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  DFS[['si_ry', 'si_by', 'si_rb']] = DFS[['si_ry', 'si_by', 'si_rb']].replace(to_replace=key, value=value, regex=True)


In [8]:
DFS

frame_timestamp  si_ry  si_by  si_rb
0                0.00      0      0      0
1                0.25      0      0      0
2                0.50      0      0      0
3                0.75      0      0      0
4                1.00      0      0      0
...               ...    ...    ...    ...
2414           603.50      0      2      2
2415           603.75      0      2      2
2416           604.00      0      2      2
2417           604.25      0      2      2
2418           604.50      0      0      0

[2419 rows x 4 columns]